In [1]:
import pandas as pd
import torch
import ast
import os
import utils

DIRPATH_DATA = "Data"
DIRPATH_EXTRACTED_FEATURES = "extracted_features"

brands_in_csv_filename = ["ad",
"brabantsdagblad",
"destentor",
"nu",
"parool",
"trouw",
"volkskrant",]

In [2]:
data_pd = pd.concat([pd.read_csv(os.path.join(DIRPATH_DATA, f"uva-relevance-windows-{brand_in_csv_filename}.csv"), sep=";")
                     for brand_in_csv_filename in brands_in_csv_filename], ignore_index=True)

In [3]:
data_pd

,article_id,brand,publication_timestamp,publication_weekday,title,first_paragraph,full_body,num_words,num_paragraphs,author_ids,labels,main_section,subsections,user_needs,lda_topics,iptc_topics_level_0,iptc_topics_level_1,time_indications_mentioned,relevance_window_in_hours
0,237bd1f6,ad,2024-05-01 00:42:30+00:00,woensdag,Brand in portiekwoning op Hannemanstraat in De...,Woensdagnacht is brand in een woning uitgebrok...,Woensdagnacht is brand in een woning uitgebrok...,43,2,['1ad8d704-fc8a-34e8-84cc-c988f31a567c'],NaN,regio,"den-haag,112-nieuws-den-haag","{'geef_me_context': 0.0004099170910194516, 'he...",{'afval_en_milieuproblematiek': 0.002882532542...,"{'ramp, ongeval en noodsituatie incident': 0.4...",{'noodhulp': 0.4418},NaN,171.0
1,d41001b,ad,2024-05-01 00:43:33+00:00,woensdag,Auto uitgebrand op Craubeekstraat in Arnhem,De brandweer is in de nacht van dinsdag op woe...,De brandweer is in de nacht van dinsdag op woe...,60,2,['1ad8d704-fc8a-34e8-84cc-c988f31a567c'],NaN,regio,"arnhem,112-nieuws-arnhem","{'geef_me_context': 0.0003419477434363216, 'he...",{'afval_en_milieuproblematiek': 0.001912314561...,"{'ramp, ongeval en noodsituatie incident': 0.6...","{'noodhulp': 0.6815, 'ongeval en noodgeval': 0...","dinsdag,woensdag",19.0
2,a091b86,ad,2024-05-01 00:45:29+00:00,woensdag,Prinses Beatrix (86) krijgt eigen musical in n...,Productiehuis MediaDome werkt aan een musical ...,Productiehuis MediaDome werkt aan een musical ...,173,4,['b163fc4b-3173-36bb-bd76-33de20eb10b6'],NaN,show,NaN,"{'geef_me_context': 0.0003985515504609793, 'he...",{'afval_en_milieuproblematiek': 0.000874452642...,"{'kunst, cultuur, entertainment en media': 0.8...",{'kunst en vermaak': 0.8627},NaN,18.0
3,919c5e9,ad,2024-05-01 00:55:06+00:00,woensdag,Brand in auto op Hulstwede in Barendrecht,De brandweer is in de nacht van dinsdag op woe...,De brandweer is in de nacht van dinsdag op woe...,58,2,['1ad8d704-fc8a-34e8-84cc-c988f31a567c'],NaN,regio,"barendrecht,112-nieuws-barendrecht","{'geef_me_context': 0.0003894437977578491, 'he...",{'afval_en_milieuproblematiek': 0.002891846466...,"{'ramp, ongeval en noodsituatie incident': 0.7...","{'noodhulp': 0.7187, 'ongeval en noodgeval': 0...","dinsdag,woensdag",55.0
4,ba0a762,ad,2024-05-01 01:00:06+00:00,woensdag,Opinie stadsdeelbestuurder Zuidoost: ‘Hier in ...,Een nieuwe wet moest gemarginaliseerde groepen...,Een nieuwe wet moest gemarginaliseerde groepen...,504,14,['a44196e8-5c16-3698-99ff-b88e9fa68034'],opinie,regio,amsterdam,"{'geef_me_context': 0.9609498381614685, 'help_...",{'afval_en_milieuproblematiek': 0.009002815932...,"{'arbeid': 0.5058, 'wetenschap en technologie'...","{'sociale wetenschappen': 0.4122, 'werkgelegen...",NaN,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213461,1829778,volkskrant,2024-10-31 19:02:00+00:00,donderdag,"‘Dagdromer’ Zhang Yiming, oprichter ByteDance ...",Mede door de wereldwijde populariteit van TikT...,Mede door de wereldwijde populariteit van TikT...,943,26,['46e47e05-311b-31f4-a4fc-9536da793bcd'],"profiel,zhang yiming",tech,NaN,"{'geef_me_context': 0.544115424156189, 'help_m...",{'afval_en_milieuproblematiek': 0.000174406130...,"{'wetenschap en technologie': 0.8022, 'economi...",{'economische sector': 0.8583},"jaar,vandaag,maand",21.0
213462,be42e4f,volkskrant,2024-10-31 19:04:44+00:00,donderdag,"‘Optimisme’ over bestand in Libanon, maar voor...",Amerikaanse onderhandelaars hebben donderdag e...,Amerikaanse onderhandelaars hebben donderdag e...,652,17,['a2e5c7d7-c647-3ec5-bde9-2a4e2846dfa1'],nieuws,buitenland,NaN,"{'geef_me_context': 0.0008029908058233559, 'he...",{'afval_en_milieuproblematiek': 0.000227075550...,"{'conflict, oorlog en vrede': 0.866, 'politiek...","{'gewapend conflict': 0.8604, 'regering': 0.622}","dinsdag,volgende week dinsdag,donderdag,dinsda...",17.0
213463,7c14bd9,volkskrant,2024-10-31 20:56:58+00:00,donderdag,Pro-Russische zege in Georgië toont dat Europa...,Waar aspirant-lidstaten als Geörgie en Moldavi...,Waar aspi

In [4]:
def EmbedS1inE2(values, max):
       values = (values / max) * torch.pi * 2
       out = torch.stack([torch.cos(values), torch.sin(values)], dim=1)
       return out


def to_vector(x, idx):
       d = ast.literal_eval(x)
       vector = torch.zeros(len(idx))
       for topic_str in d:
              vector[idx[topic_str]] = d[topic_str]
       return vector


def to_ohe(column):
       i2s = column.unique().tolist()
       s2i = {s: i for i, s in enumerate(i2s)} 

       one_hot_encoder = torch.eye(len(i2s))
       column_index = [s2i[row] for row in column]
       
       return one_hot_encoder[column_index]


def to_brand_dir(feature_tensor, indices_dict, filename):
       for brand in indices_dict:
              dir_path_brand = os.path.join(DIRPATH_EXTRACTED_FEATURES, brand)
              if not os.path.exists(dir_path_brand):
                     os.makedirs(dir_path_brand)
              file_path = os.path.join(dir_path_brand, filename)
              torch.save(feature_tensor[indices_dict[brand]], file_path)

In [5]:
brands = data_pd["brand"].unique().tolist()

brand_indices = {brand: torch.tensor(data_pd.index[data_pd['brand'] == brand]).int() for brand in brands}

In [ ]:
brand_ohe = to_ohe(data_pd["brand"])
to_brand_dir(brand_ohe, brand_indices, utils.FILENAME_BRAND_OHE)

minutes = []
for row in data_pd["publication_timestamp"]:
       minutes.append(int(row[11:13]) * 60 + int(row[14:16]))
minutes = torch.tensor(minutes)

to_brand_dir(minutes.unsqueeze(1), brand_indices, utils.FILENAME_PUBLICATION_TIMESTAMP)

minutes_S1 = EmbedS1inE2(minutes, 60*24)
to_brand_dir(minutes_S1, brand_indices, utils.FILENAME_PUBLICATION_TIMESTAMP_EMB)

weekday_ohe = to_ohe(data_pd["publication_weekday"])
to_brand_dir(weekday_ohe, brand_indices, utils.FILENAME_PUBLICATION_WEEKDAY_OHE)

num_word = torch.tensor(data_pd['num_words']).unsqueeze(1)
to_brand_dir(num_word, brand_indices, utils.FILENAME_NUM_WORDS)

num_par = torch.tensor(data_pd['num_words']).unsqueeze(1)
to_brand_dir(num_par, brand_indices, utils.FILENAME_NUM_PARAGRAPH)


main_ohe = to_ohe(data_pd['main_section'])
to_brand_dir(main_ohe, brand_indices, utils.FILENAME_MAIN_SECTION_OHE)

subsections_ohe = to_ohe(data_pd['subsections'])
to_brand_dir(subsections_ohe, brand_indices, utils.FILENAME_SUBSECTIONS_OHE)

user_needs = torch.tensor(data_pd["user_needs"].map(lambda user_needs: list(ast.literal_eval(user_needs).values())))
to_brand_dir(user_needs, brand_indices, utils.FILENAME_USER_NEEDS)

lda_topics = torch.tensor(data_pd["lda_topics"].map(lambda lda_topics: list(ast.literal_eval(lda_topics).values())))
to_brand_dir(lda_topics, brand_indices, utils.FILENAME_LDA_TOPICS)

iptc_level_0 = set()
for row in data_pd["iptc_topics_level_0"]:
       iptc_level_0 = iptc_level_0.union(set(ast.literal_eval(row).keys()))
iptc_level_1 = set()
for row in data_pd["iptc_topics_level_1"]:
       iptc_level_1 = iptc_level_1.union(set(ast.literal_eval(row).keys()))
iptc_level_0_indexed = {topic_str: topic_id for topic_id, topic_str in enumerate(iptc_level_0)}
iptc_level_1_indexed = {topic_str: topic_id for topic_id, topic_str in enumerate(iptc_level_1)}

iptc_0 = torch.stack(data_pd['iptc_topics_level_0'].map(lambda x: to_vector(x, iptc_level_0_indexed)).tolist())
to_brand_dir(iptc_0, brand_indices, utils.FILENAME_IPTC_TOPICS_LEVEL_0)

iptc_1 = torch.stack(data_pd['iptc_topics_level_1'].map(lambda x: to_vector(x, iptc_level_1_indexed)).tolist())
to_brand_dir(iptc_1, brand_indices, utils.FILENAME_IPTC_TOPICS_LEVEL_1)

target = torch.FloatTensor(data_pd['relevance_window_in_hours']).unsqueeze(1)
to_brand_dir(target, brand_indices, utils.FILENAME_RELEVANCE_WINDOW)


